# Start Interactive Dask Cluster

In [1]:
# this is for a strange bug with compute instances 
import os

os.system('sudo cp /etc/nginx/nginx.conf setup/temp.conf') # stupid

nginx = ''

with open('setup/temp.conf') as f:
    for line in f.readlines():
        if 'websocket/|/ws/' in line:
            nginx += line.replace('websocket/|/ws/', 'websocket/|/ws')
        else:
            nginx += line
       
with open('setup/temp2.conf', 'w') as f:
    f.write(nginx)
    
os.system('sudo mv setup/temp2.conf /etc/nginx/nginx.conf')
os.system('sudo service nginx restart')
os.system('rm setup/temp.conf');

## Imports

Import all packages used in this notebook.

In [2]:
import os
import time
import socket

from IPython.core.display import HTML

from azureml.widgets import RunDetails
from azureml.train.estimator import Estimator
from azureml.core.runconfig import MpiConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Workspace, Experiment, Datastore, Dataset, Environment

%matplotlib inline

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception cannot import name '_DistributedTraining'.


## Azure ML setup

Get the workspace.

In [3]:
ws = Workspace.from_config()
ws

Workspace.create(name='azureml-dask', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters-rg')

### Enter your name

Enter your name and virtual network information.

In [4]:
### name
name        = 'dask'             # REPLACE

### vnet settings
vnet_rg     = ws.resource_group  # replace if needed
vnet_name   = 'uksouth-vnet'     # replace if needed
subnet_name = 'default'          # replace if needed

### azure ml names 
ct_name     = f'{name}-dask-ct'
exp_name    = f'{name}-dask-demo'

### trust but verify
verify = f'''
Name: {name}

vNET RG: {vnet_rg}
vNET name: {vnet_name}
vNET subnet name: {subnet_name}

Compute target: {ct_name}
Experiment name: {exp_name}
'''

print(verify)


Name: dask

vNET RG: copeters-rg
vNET name: uksouth-vnet
vNET subnet name: default

Compute target: dask-dask-ct
Experiment name: dask-dask-demo



### Create VM pool

Create Azure ML VM pool for creating remote dask cluster(s).

In [5]:
if ct_name not in ws.compute_targets:
    # create config for Azure ML cluster
    # change properties as needed
    config = AmlCompute.provisioning_configuration(
             vm_size                       = 'STANDARD_D14_V2',  
             min_nodes                     = 0,
             max_nodes                     = 200,
             vnet_resourcegroup_name       = vnet_rg,              
             vnet_name                     = vnet_name,            
             subnet_name                   = subnet_name,          
             idle_seconds_before_scaledown = 300
    )
    ct = ComputeTarget.create(ws, ct_name, config)
    ct.wait_for_completion(show_output=True)    
else:
    ct = ws.compute_targets[ct_name]
    
ct

AmlCompute(workspace=Workspace.create(name='azureml-dask', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copeters-rg'), name=dask-dask-ct, id=/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/copeters-rg/providers/Microsoft.MachineLearningServices/workspaces/azureml-dask/computes/dask-dask-ct, type=AmlCompute, provisioning_state=Succeeded, location=uksouth, tags=None)

## Startup cluster

Start the run now. The first time, this will take 

In [6]:
codefileshare = 'codefileshare'

if codefileshare not in ws.datastores:
    Datastore.register_azure_file_share(ws, codefileshare,
                                        'code-391ff5ac-6576-460f-ba4d-7e03433c68b6',                     # stupid
                                        account_name = ws.datastores['workspacefilestore'].account_name, # less stupid
                                        account_key  = ws.datastores['workspacefilestore'].account_key   # less less stupid
                                       )

In [7]:
daskdata = 'noaa-isd-files'

if daskdata not in ws.datasets:
    ds = Dataset.File.from_files('https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather/*/*/*.parquet', validate=False)
    os.system('sudo chmod 777 /mnt')
    ds.download('/mnt/data/isd')
    ws.datastores['gen2'].upload('/mnt/data/isd', '/noaa-isd')
    ds = ds.register(ws, daskdata)

In [8]:
# # of nodes 
nodes = 10; GPU = False  

script_params = {
    '--jupyter': True,
    '--code_store': ws.datastores[codefileshare], 
    '--data_store': ws.datastores['gen2']         # replace with relevant datastore
}

if GPU:
    script_params['--use_GPU']         = True
    script_params['--n_gpus_per_node'] = 4
    env_params = {'custom_docker_image': 'todrabas/aml_rapids:latest', 'user_managed': True}
else:
    env_params = {'pip_packages': ['mpi4py',
                                   'distributed',
                                   'dask[complete]',
                                   'dask-ml[complete]',
                                   'fastparquet',
                                   'pyarrow',
                                   'jupyterlab',
                                   'joblib',
                                   'notebook',
                                   'adlfs', 
                                   'fsspec', 
                                   'azureml-sdk[notebooks]',
                                   'azureml-dataprep[fuse]',
                                   'lz4']}

exp   = Experiment(ws, exp_name)
est   = Estimator('setup', 
                  compute_target          = ct, 
                  entry_script            = 'start.py',          # sets up Dask cluster
                  script_params           = script_params,
                  node_count              = nodes,        
                  distributed_training    = MpiConfiguration(),
                  **env_params
)

#### SET PROPER INTERPRETER - from Tom
if GPU:
    est._estimator_config.environment.python.interpreter_path = '/opt/conda/envs/rapids/bin/python' 

#run = next(exp.get_runs()) # use this to get existing run (if kernel restarted, etc)
run = exp.submit(est); run.log('nodes', nodes)
run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-dask-demo,dask-dask-demo_1580677731_4cf570fc,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Jupyter/Dashboard setup

In [10]:
# port to forward the dask dashboard to on the compute instance
# we do not use 8787 because it is already in use 
dashboard_port = 4242
jupyter_port   = 9999

print("waiting for scheduler node's ip")
while run.get_status() != 'Canceled' and 'scheduler' not in run.get_metrics():
    print('.', end ="")
    time.sleep(5)

if run.get_status() == 'Canceled':
    print('\nRun was canceled')
else:
    print(f'\nSetting up ports...')
    os.system(f'killall socat') # kill all socat processes - cleans up previous port forward setups 
    os.system(f'setsid socat tcp-listen:{dashboard_port},reuseaddr,fork tcp:{run.get_metrics()["dashboard"]} &')
    os.system(f'setsid socat tcp-listen:{jupyter_port},reuseaddr,fork tcp:{run.get_metrics()["jupyter"]} &')
    print(f'Done')

waiting for scheduler node's ip
....
Setting up ports...
Done


In [11]:
# build the dashboard link 
dashboard_url = f'https://{socket.gethostname()}-{dashboard_port}.{ws.get_details()["location"]}.instances.azureml.net/status'
HTML(f'<a href="{dashboard_url}">Dashboard link</a>')

In [12]:
# build the jupyter link 
jupyter_url = f'https://{socket.gethostname()}-{jupyter_port}.{ws.get_details()["location"]}.instances.azureml.net/lab?token={run.get_metrics()["token"]}'
HTML(f'<a href="{jupyter_url}">Jupyter link</a>')

## Complete the run

In [13]:
#run.cancel(); run.complete()